In [1]:
import sys

In [2]:
sys.path.append("../pseudosplit")

In [3]:
import numpy as np

In [4]:
import matplotlib.pyplot as plt
%matplotlib qt

In [5]:
from basis import FourierBasis
from state import State
from scheme import (Strang, Neri, Yoshida6,
                    AffineSym2, AffineSym4, AffineSym6, AffineSym8) 

In [6]:
from solver import Solver

In [7]:
from models import NLSE3Model1D

In [8]:
# List of splitting methods
methods_list = [Strang(), Neri(), Yoshida6(),
                AffineSym2(), AffineSym4(), AffineSym6()]

In [9]:
dt_list = [1e-1, 5.0e-2, 2.5e-2, 1.0e-2, 5e-3, 2.5e-3]

In [10]:
# Soliton parameters
eta = 1.0  # amplitude
c = 0.01  # speed
omega = 0.5*(c**2 - eta**2)  # angular frequency

# Define the soliton function
soliton = lambda x: eta / np.cosh(eta*x) * np.exp(1j*c*x)

In [11]:
# Set the pseudo-spectral basis
N = 2**11  # Number of Fourier modes
I = (-50*np.pi, 50*np.pi)  # Interval
fb = FourierBasis('fb', N, I)

In [12]:
# Define the initial state
u0 = State(name='u0', basis=fb, u=soliton) 

In [13]:
# Set the model
model = NLSE3Model1D()
H = model.get_hamiltonian(fb)

In [14]:
plt.style.use("classic")
fig_H, ax_H = plt.subplots()
fig_M, ax_M = plt.subplots()

ax_H.grid()
ax_M.grid()

In [15]:
for method in methods_list:
    for dt in dt_list:
        print(method.name, dt)
        # Create the solver
        solver = Solver(model, method)

        # Initialize solver
        t0 = 0.0
        tf = 1000.0
        solver.start(u0, t0, tf) 

        # First point of trajectory
        t_values = [t0]
        H_values = [H(u0)]
        M_values = [u0.norm()]

        count = 0
        while solver.active:
            u = solver.step(dt)
            t = solver.sim_time
            t_values.append(t)
            H_values.append(H(u))
            M_values.append(u.norm()) 
            count += 1
            if count % 10000 == 0:
                print(t)
                count = 0
        H_rel_err = np.abs((H_values-H_values[0])/H_values[0])
        M_rel_err = np.abs((M_values-M_values[0])/M_values[0])
        print(method.P_A.nfev)
        label = "$\mathtt{" + method.name + "}, \Delta t=" + str(dt) + "$"
        ax_H.plot(t_values, H_rel_err, label=label)
        ax_M.plot(t_values, M_rel_err, label=label)
        fig_H.show()
        fig_M.show()
        
        

yoshida6 0.1
1000.0
80000
yoshida6 0.05
500.0000000000794
999.9999999996382
160008
yoshida6 0.025
250.0000000000397
499.9999999998191
749.9999999995918
999.9999999993644
320008
yoshida6 0.01
100.00000000001425
199.99999999996308
299.99999999987216
399.9999999997812
499.99999999969026
599.9999999995994
699.9999999995084
799.9999999994175
899.9999999993265
999.9999999992356
800008
yoshida6 0.005
50.00000000000713
99.99999999998154
149.99999999993608
199.9999999998906
249.99999999984513
299.9999999997997
349.9999999997542
399.99999999970873
449.99999999966326
499.9999999996178
549.9999999995723
599.9999999995268
649.9999999994814
699.9999999994359
749.9999999993904
799.9999999993449
849.9999999992995
899.999999999254
949.9999999992085
999.999999999163
1600008
yoshida6 0.0025
25.000000000003563
49.99999999999077
74.99999999996804
99.9999999999453
124.99999999992257
149.99999999989984
174.9999999998771
199.99999999985437
224.99999999983163
249.9999999998089
274.99999999978616
299.9999999997

In [16]:
ax_H.legend(loc="upper left")
ax_M.legend(loc="upper left")